# Final Project - Dashboard

**Name:** Omar Alejandro Guzmán Munguía

**e-mail:** omar.guzman5063@alumnos.udg.mx

# MODULES

In [3]:
import panel as pn
pn.extension()

import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import panel.widgets as pnw
pn.extension('plotly')

# FUNCTIONS

In [ ]:
def load_data(results_file_path):
    """
    Load data from a JSON file and return it as a dictionary.
    Args:
        results_file_path (str): Path to the JSON file.
    Returns:
        dict: Data loaded from the JSON file.
    """
    try:
        with open(results_file_path, 'r') as file:
            return json.load(file)
    except Exception as e:
        print(f"Error loading data: {e}")

In [6]:
def extract_model_names(json_path):
    """
    Extract unique model names from a JSON file
    
    Args:
        json_path (str or dict): Path to the JSON file
        
    Returns:
        list: A list containing unique model names.
    """
    try:
        with open(json_path, 'r') as file:
            data = json.load(file)
            
        unique_models = set()
        
        for category in data:
            for model in category.get("models", []):
                if "model" in model:
                    model_name = model["model"]
                    
                    if "size" in model and model["size"] != "N/A":
                        unique_identifier = f"{model_name} {model['size']}"
                    else:
                        unique_identifier = model_name
                        
                    unique_models.add(unique_identifier)
        
        return list(unique_models)
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return []

In [7]:
def extract_categories(json_path):
    """
    Extract category names from a JSON file.
    
    Args:
        json_path (str or dict): Path to the JSON file
        
    Returns:
        list: A list containing all category names.
    """
    try:
        with open(json_path, 'r') as file:
            data = json.load(file)
        
        category_names = [category.get("category", "") for category in data if "category" in category]
        
        return category_names
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return []

In [8]:
def extract_metrics_name(json_path):
    """
    Extract unique metrics names from a JSON file.
    
    Args:
        json_path (str or dict): Path to the JSON file
        
    Returns:
        list: A list containing unique metrics names.
    """
    try:
        with open(json_path, 'r') as file:
            data = json.load(file)
            
        metric_keys = set()
        for key in data[0]["models"][0]:
            if (key.startswith(("pass@", "recall@"))):
                metric_keys.add(key)
        
        return sorted(metric_keys)
    
    except Exception as e:
        print(f"Error: {str(e)}")
        return []

In [ ]:
def create_scatter_plot(df, x_metric_name, y_metric_name):
    """
    Create a scatter plot using Plotly Express.
    Args:
        df (pd.DataFrame): DataFrame containing the data to plot.
        x_metric_name (str): Name of the metric for the x-axis.
        y_metric_name (str): Name of the metric for the y-axis.
    Returns:
        plotly.graph_objects.Figure: A Plotly figure object.
    """
    df['model_with_size'] = df.apply(lambda x: f"{x['model']} ({x['size']})" if x['size'] != 'N/A' else x['model'], axis=1)
    
    fig = px.scatter(
        df, 
        x=x_metric_name, 
        y=y_metric_name, 
        color='model_with_size',
        hover_data=['model', 'size'],
        title=f'Model Performance: {x_metric_name} vs. {y_metric_name}'
    )
    
    fig.update_layout(
        xaxis_title=x_metric_name,
        yaxis_title=y_metric_name,
        height=500
    )
    
    return fig

# INITIALIZATION

In [ ]:
# repositories_list = extract_repository_names('data/repositories.json')
# default_repo = repositories_list[0] if repositories_list else None

models_list = extract_model_names('data/results.json')
default_model = models_list[0] if models_list else None

templates_list = extract_categories('data/results.json')
default_template = templates_list[0] if templates_list else None

metrics_list = extract_metrics_name('data/results.json')
default_metric = metrics_list[0] if metrics_list else None

In [ ]:
def create_heatmap(df):
    """
    Create a heatmap using Plotly.
    Args:
        df (pd.DataFrame): DataFrame containing the data to plot.
    Returns:
        plotly.graph_objects.Figure: A Plotly figure object.
    """
    df['model_with_size'] = df.apply(lambda x: f"{x['model']} ({x['size']})" if x['size'] != 'N/A' else x['model'], axis=1)
    
    pivot_df = df.pivot_table(index='model_with_size', values=metrics_list)
    
    fig = go.Figure(data=go.Heatmap(
        z=pivot_df.values,
        x=pivot_df.columns,
        y=pivot_df.index,
        colorscale='Viridis',
        colorbar=dict(title='Value'),
        hoverongaps=False
    ))
    
    fig.update_layout(
        title='Model Performance Heatmap',
        xaxis_title='Metrics',
        yaxis_title='Models',
        height=500
    )
    
    return fig

In [ ]:
def prepare_data(data, template_value):
    """
    Prepare data for visualization by filtering based on the template value.
    Args:
        data (list): List of dictionaries containing model performance data.
        template_value (str): Template value to filter the data.
    Returns:
        pd.DataFrame: DataFrame containing the filtered data.
    """
    filtered_data = [entry for entry in data if entry.get('category') == template_value]
    
    # List to collect all model performance data
    all_model_data = []
    
    for entry in filtered_data:
        for model_data in entry.get('models', []):
            model_entry = {
                'model': model_data.get('model', 'Unknown'),
                'size': model_data.get('size', 'N/A')
            }
            # Add metrics
            for metric in metrics_list:
                if metric in model_data:
                    model_entry[metric] = model_data[metric]
            
            all_model_data.append(model_entry)
    
    return pd.DataFrame(all_model_data)

# WIDGETS

In [24]:
# repositories = pnw.Select(name='Repository', width=160, options=repositories_list, value=default_repo)
metrics = pnw.Select(name='Metric', width=160, options=metrics_list, value=default_metric)
models = pnw.Select(name='Model', width=160, options=models_list, value=default_model)
template = pnw.Select(name='Template', width=160, options=templates_list, value=default_template)
plot_type = pnw.Select(name='Plot Type', width=160, options=['Heatmap', 'Scatter Plot'], value='Heatmap')

x_metric = pnw.Select(name='X-Axis Metric', width=160, options=metrics_list, value=metrics_list[0])
y_metric = pnw.Select(name='Y-Axis Metric', width=160, options=metrics_list, value=metrics_list[4] if len(metrics_list) > 4 else metrics_list[-1])

In [ ]:
@pn.depends(template, x_metric, y_metric, plot_type)
def create_visualization(template_value, x_metric_value, y_metric_value, viz_type):
    """
    Create a visualization based on the selected template and metrics.
    Args:
        template_value (str): Selected template value.
        x_metric_value (str): Selected x-axis metric.
        y_metric_value (str): Selected y-axis metric.
        viz_type (str): Type of visualization ('Heatmap' or 'Scatter Plot').
    Returns:
        panel.pane.Plotly: A Plotly pane containing the visualization.
    """
    data = load_data('data/results.json')
    df = prepare_data(data, template_value)
    
    if viz_type == 'Heatmap':
        fig = create_heatmap(df)
    else:  
        fig = create_scatter_plot(df, x_metric_value, y_metric_value)
    
    return pn.pane.Plotly(fig)

# PANEL

In [ ]:
description = pn.pane.Markdown("""
## Model Performance Visualization
Select parameters below to visualize model performance across different metrics.
- **Heatmap**: Shows performance of all models across all metrics
- **Scatter Plot**: Compares two specific metrics across models
""")

# Create the layout for the dashboard
controls = pn.Column(
    template,
    plot_type,
    x_metric,
    y_metric,
    width=200
)

# Create the dashboard layout
dashboard = pn.Column(
    description,
    pn.Row(controls, create_visualization),
    width=800
)

# Serve the dashboard
dashboard.servable()

Column(width=800)
    [0] Markdown(str)
    [1] Row
        [0] Column(width=200)
            [0] Select(name='Template', options=['Local File (Infilling)',...], value='Local File (Infilling)', width=160)
            [1] Select(name='Plot Type', options=['Heatmap', 'Scatter P...], value='Heatmap', width=160)
            [2] Select(name='X-Axis Metric', options=['pass@1', 'pass@10', ...], value='pass@1', width=160)
            [3] Select(name='Y-Axis Metric', options=['pass@1', 'pass@10', ...], value='recall@1', width=160)
        [1] ParamFunction(function, _pane=Plotly, defer_load=False)